## **Load Packages and Set Environment Variables**

In [ ]:
import numpy as np
import pandas as pd

import os
import cv2
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split

import time
import torch
import zipfile
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from datetime import datetime


DRIVE = "/content/Larch"
IMAGE_ZIP = "Data_Set_Larch_Casebearer.zip"
IMAGES = "Data_Set_Larch_Casebearer"
os.environ["DRIVE"] = DRIVE
os.environ["DRIVE_ZIP"] = f"{DRIVE}/{IMAGE_ZIP}"
os.environ["IMAGE_ZIP"] = IMAGE_ZIP
os.environ["IMAGES"] = IMAGES

BASE_DIR=DRIVE

IMG_SIZE = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
BATCH_SIZE = 2
D_SIZE = 1  # Model Size
C_SIZE = 1  # Image Size

## **Load Data and Install Petrel**

Petrel streamlines the model pipeline.

In [ ]:
%%bash
wget https://lilablobssc.blob.core.windows.net/larch-casebearer/$IMAGE_ZIP
unzip -q /content/$IMAGE_ZIP
rm /content/$IMAGE_ZIP

pip install -U -q albumentations
pip install -q petrel-det

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from petrel.dataset import TrainDataset, ValDataset
from petrel.model import load_edet, load_optimizer, load_scheduler, ModelTrainer

## Bounding Box Parameters.
BBOX = A.BboxParams(format='pascal_voc',
                    min_area=0,
                    min_visibility=0,
                    label_fields=['labels'])

In [ ]:
def xml_to_df(d, directory, filename):
    xml_split = filename.split('/')
    file = xml_split[-1].split('.')[0]
    location, date = directory.split("/")[1].split('_')
    root = ET.XML(d)
    objects = []
    for child in root:
        if child.tag == 'object':
            objects.append(child)
        elif child.tag == 'size':
            dims = {s.tag: int(s.text) for s in child}
    
    datas = []
    for child in objects:
        data ={}
        for c in child:
            if c.tag != 'bndbox':
                data[c.tag] = [c.text]
            else:
                for b in c:
                    data[b.tag] = [int(b.text)]
        datas.append(pd.DataFrame(data))
    try:
        df = pd.concat(datas)
        df['height'], df['width'] = dims['height'], dims['width']
        df['location'], df['date'], df['file_name'] = location, int(date), file
    except:
        print(filename)
        df = pd.DataFrame({'height': dims['height'],
                           'width': dims['width'],
                           'location': [location],
                           'date': [int(date)],
                           'file_name': [file]})
    df["file"] = df.apply(lambda row: f"{IMAGES}/{row['location']}_{row['date']}/Images/{row['file_name']}.JPG", axis=1)
    
    return df

def read_xml(xml_dir):
    df_list = []
    for filename in [f"{IMAGES}/{xml_dir}/Annotations/{f}" for  f in os.listdir(f"{IMAGES}/{xml_dir}/Annotations") if "__" not in f and ".xml" in f]:
        with open(filename) as f:
            d = f.read() 
            df_list.append(xml_to_df(d, f"{IMAGES}/{xml_dir}/Annotations", filename))
    
    return pd.concat(df_list).reset_index(drop=True)

# **Process XML metadata to DataFrame**

In [ ]:
%%time
def get_meta_data():
  larch_dirs = [f for f in os.listdir(IMAGES)]
  image_cols = ["location", "file", "file_name", "height", "width"]
  box_cols = ["tree", "damage", "labels", "xmin", "ymin", "xmax", "ymax", "file"]

  ## Concatenate Dataframes from each location into single Dataframe.
  xml_df = pd.concat([read_xml(ld) for ld in larch_dirs]).reset_index(drop=True)
  xml_df['truncated'] = xml_df['truncated'].astype(float)
  
  ## Standardize column names.
  xml_df.loc[xml_df[~xml_df['name'].isna()].index, 'tree'] = xml_df.loc[xml_df[~xml_df['name'].isna()].index, 'name']

  ## Drop redundant columns.
  xml_df.drop(columns=['name', 'difficult', 'pose'], inplace=True)

  ## Standardize label for detection of other tree species.
  xml_df.loc[xml_df[xml_df['tree'].isna()].index, 'tree'] = 'Other'
  xml_df['tree'] = xml_df['tree'].apply(lambda t: t.capitalize()).apply(lambda t: t if t != 'Spruce' else 'Other')

  ## Remove detections with no damage information.
  xml_df = xml_df[~xml_df['damage'].isnull()].reset_index(drop=True)
  xml_df['truncated'] = xml_df['truncated'].astype(int)
  for col in ["truncated", "xmin", "xmax", "ymin", "ymax"]:
    xml_df[col] = xml_df[col].astype(int)
  damage_map = {d: n + 1 for n, d in enumerate(xml_df["damage"].sort_values().unique())}
  xml_df["labels"] = xml_df["damage"].apply(lambda d: damage_map[d])
  xml_df = xml_df[xml_df['file_name'] != "B01_0023"]

  ## Drop duplicate entries.
  xml_df = xml_df.drop_duplicates().reset_index(drop=True)

  ## Remove August Data.
  xml_images = xml_df[xml_df['date'] == 20190527][image_cols].drop_duplicates().reset_index(drop=True)
  xml_boxes = xml_df[xml_df['date'] == 20190527][box_cols].reset_index(drop=True)
  xml_boxes = xml_boxes[xml_boxes["xmin"] != xml_boxes["xmax"]].reset_index(drop=True)

  # Train-Val Split
  xml_train, xml_val = train_test_split(xml_images,
                                        test_size=0.2,
                                        random_state=64,
                                        stratify=xml_images['location'])
  xml_train.reset_index(drop=True, inplace=True)
  xml_val.reset_index(drop=True, inplace=True)
  xml_train_boxes = xml_boxes[xml_boxes["file"].isin(xml_train["file"])].reset_index(drop=True)
  xml_val_boxes = xml_boxes[xml_boxes["file"].isin(xml_val["file"])].reset_index(drop=True)
  return xml_train, xml_val, xml_train_boxes, xml_val_boxes

xml_train, xml_val, xml_train_boxes, xml_val_boxes = get_meta_data()

**Training Preprocessing**

Random crop.

Random horizontal and vertical flips.
Random rotation and transpose.

**Validation Preprocessing**

Resize

In [ ]:
def get_train_transforms(d_size):
    """
    Returns a function to perform the standard sequence of preprocessing steps
    for training data.
    """
    return A.Compose([A.RandomResizedCrop(height=IMG_SIZE[d_size],
                                          width=IMG_SIZE[d_size],
                                          scale=(0.05, 1),
                                          ratio=(3/4, 4/3),
                                          p=1.0),
                      A.HorizontalFlip(p=0.5),
                      A.VerticalFlip(p=0.5),
                      A.RandomRotate90(p=1.0),
                      A.Transpose(p=0.5),
                      ToTensorV2(p=1.0)],
                     bbox_params=BBOX,
                     p=1.0)

def get_val_full_transform(d_size):
    """
    Returns a function to perform the standard sequence of preprocessing steps
    for validation data.
    """
    return A.Compose([A.Resize(height=IMG_SIZE[d_size],
                               width=IMG_SIZE[d_size],
                               p=1.0),
                      ToTensorV2(p=1.0)],
                     bbox_params=BBOX,
                     p=1.0)

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataset = TrainDataset(meta_data=xml_train,
                             boxes=xml_train_boxes,
                             image_root="/content",
                             transform=get_train_transforms(d_size=D_SIZE))
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=RandomSampler(train_dataset),
    pin_memory=False,
    drop_last=True,
    collate_fn=collate_fn)

val_full_dataset = ValDataset(
    meta_data=xml_val,
    boxes=xml_val_boxes,
    image_root="/content",
    transform=get_val_full_transform(d_size=D_SIZE),
    train_pipe=True
)

val_full_loader = torch.utils.data.DataLoader(
    val_full_dataset, 
    batch_size=BATCH_SIZE,
    sampler=SequentialSampler(val_full_dataset),
    shuffle=False,
    pin_memory=False,
    collate_fn=collate_fn)

**Set up model**

200 Epochs

Initial learning rate 0.000256

Cosine decay to 0 over all Epochs.

In [ ]:
N = 200
model = load_edet(f"tf_efficientdet_d{D_SIZE}", image_size=IMG_SIZE[D_SIZE],
                  num_classes=4)
optimizer = load_optimizer("adamw", model, learning_rate=2.56e-4)
scheduler = load_scheduler("cosine", optimizer=optimizer, T_max=N)
model_trainer = ModelTrainer(model, optimizer, scheduler,
                             base_dir=f"{DRIVE}/effdet{D_SIZE}_petrel_cosine256e4_bs4",
                             verbose_step=20,
                             num_epochs=N)

In [ ]:
model_trainer.fit(train_loader, val_full_loader)